# Visualizing mbuild Compound

In this example, we will use `fresnel` to visualize an mbuild Compound. We will color the particles by types, as well as visualize the simulation box.

In [ ]:
from PySide2 import QtCore
%gui qt

In [ ]:
import fresnel
import fresnel.interact
import numpy as np
import mbuild as mb
import freud

In [ ]:
def mb_to_freud_box(box):
    """  
    Convert an mbuild box object to a freud box object
    These sites are helpful as reference: 
    http://gisaxs.com/index.php/Unit_cell
    https://hoomd-blue.readthedocs.io/en/stable/box.html

    Parameters
    ----------
    box : mbuild.box.Box()

    Returns
    -------
    freud.box.Box()
    """
    Lx = box.lengths[0]
    Ly = box.lengths[1]
    Lz = box.lengths[2]
    alpha = box.angles[0]
    beta = box.angles[1]
    gamma = box.angles[2]

    frac = (
        (np.cos(np.radians(alpha)) - np.cos(np.radians(beta)) * np.cos(np.radians(gamma))) 
        / np.sin(np.radians(gamma))
    )
    c = np.sqrt(1 - np.cos(np.radians(beta))**2 - frac**2)
    
    xy = np.cos(np.radians(gamma)) / np.sin(np.radians(gamma))
    xz = frac / c
    yz = np.cos(np.radians(beta)) / c
    
    box_list = list(box.maxs) + [xy, yz, xz]
    return freud.box.Box(*box_list)


class Methane(mb.Compound):
    def __init__(self):
        super(Methane, self).__init__()
        carbon = mb.Particle(name='C')
        self.add(carbon, label='C[$]')

        hydrogen = mb.Particle(name='H', pos=[0.1, 0, -0.07])
        self.add(hydrogen, label='HC[$]')
        
        self.add_bond((self[0], self['HC'][0]))

        self.add(mb.Particle(name='H', pos=[-0.1, 0, -0.07]), label='HC[$]')
        self.add(mb.Particle(name='H', pos=[0, 0.1, 0.07]), label='HC[$]')
        self.add(mb.Particle(name='H', pos=[0, -0.1, 0.07]), label='HC[$]')

        self.add_bond((self[0], self['HC'][1]))
        self.add_bond((self[0], self['HC'][2]))
        self.add_bond((self[0], self['HC'][3]))

methane = Methane()
#methane.visualize().show()

box = mb.Box(lengths=[0.5,0.5,0.5])

In [ ]:
cpk_colors = np.array(
    [
        [1.00, 1.00, 1.00], # white
        [0.30, 0.30, 0.30], # grey
        [0.13, 0.20, 1.00], # dark blue
        [1.00, 0.13, 0.00], # red
        [0.12, 0.94, 0.12], # green
        [0.60, 0.13, 0.00], # dark red
        [0.40, 0.00, 0.73], # dark violet
        [0.00, 1.00, 1.00], # cyan
        [1.00, 0.60, 0.00], # orange
        [1.00, 0.90, 0.13], # yellow
        [1.00, 0.67, 0.47], # peach
        [0.47, 0.00, 1.00], # violet
        [0.00, 0.47, 0.00], # dark green
        [0.60, 0.60, 0.60], # grey
        [0.87, 0.47, 0.00], # dark orange
        [0.87, 0.47, 1.00]  # pink
    ]
)


In [ ]:
N = methane.n_particles
particle_names = [p.name for p in methane.particles()]

color_dict = {
    "H": fresnel.color.linear(cpk_colors[0]),
    "C": fresnel.color.linear(cpk_colors[1]),
    "N": fresnel.color.linear(cpk_colors[2]),
    "O": fresnel.color.linear(cpk_colors[3]),
    "F": fresnel.color.linear(cpk_colors[4]),
    "Cl": fresnel.color.linear(cpk_colors[4]),
    "Br": fresnel.color.linear(cpk_colors[5]),
    "I": fresnel.color.linear(cpk_colors[6]),
    "He": fresnel.color.linear(cpk_colors[7]),
    "Ne": fresnel.color.linear(cpk_colors[7]),
    "Ar": fresnel.color.linear(cpk_colors[7]),
    "Xe": fresnel.color.linear(cpk_colors[7]),
    "Kr": fresnel.color.linear(cpk_colors[7]),
    "P": fresnel.color.linear(cpk_colors[8]),
    "S": fresnel.color.linear(cpk_colors[9]),
    "B": fresnel.color.linear(cpk_colors[10]),
    "Li": fresnel.color.linear(cpk_colors[11]),
    "Na": fresnel.color.linear(cpk_colors[11]),
    "K": fresnel.color.linear(cpk_colors[11]),
    "Rb": fresnel.color.linear(cpk_colors[11]),
    "Cs": fresnel.color.linear(cpk_colors[11]),
    "Fr": fresnel.color.linear(cpk_colors[11]),
    "Be": fresnel.color.linear(cpk_colors[12]),
    "Mg": fresnel.color.linear(cpk_colors[12]),
    "Ca": fresnel.color.linear(cpk_colors[12]),
    "Sr": fresnel.color.linear(cpk_colors[12]),
    "Ba": fresnel.color.linear(cpk_colors[12]),
    "Ra": fresnel.color.linear(cpk_colors[12]),
    "Ti": fresnel.color.linear(cpk_colors[13]),
    "Fe": fresnel.color.linear(cpk_colors[14]),
}

# Made space to add more later
radii_dict = {
    "H": 0.05,
    "default": 0.06
}


# all_bonds.shape is (nbond, 2 ends, xyz)
all_bonds = np.stack([np.stack((i[0].pos, i[1].pos)) for i in methane.bonds()])


N_bonds = all_bonds.shape[0]

freud_box = mb_to_freud_box(box)

# Make an array of the colors based on particle name 
# -- if name is not defined in the dictionary, use pink
color_array = np.empty((N,3), dtype="float64")
for i,n in enumerate(particle_names):
    try:
        color_array[i,:] = color_dict[n]
    except KeyError:
        color_array[i,:] = fresnel.color.linear(cpk_color[-1])

# Make an array of the radii based on particle name 
# -- if name is not defined in the dictionary, use default
rad_array = np.empty((N), dtype="float64")
for i,n in enumerate(particle_names):
    try:
        rad_array[i] = radii_dict[n]
    except KeyError:
        rad_array[i] = radii_dict["default"]

In [ ]:
scene = fresnel.Scene()

# Spheres for every particle in the system
geometry = fresnel.geometry.Sphere(scene, N=N, radius=0.06)
geometry.position[:] = methane.xyz
geometry.material = fresnel.material.Material(roughness=1.0)
geometry.outline_width = 0.01

# use color instead of material.color
geometry.material.primitive_color_mix = 1.0
geometry.color[:] = color_array

# resize radii
geometry.radius[:] = rad_array

# bonds
bonds = fresnel.geometry.Cylinder(scene, N=N_bonds)
bonds.material = fresnel.material.Material(roughness=0.5)
bonds.outline_width = 0.01

# bonds are white
bond_colors = np.ones((N_bonds, 3), dtype="float64")

bonds.material.primitive_color_mix = 1.0
bonds.points[:] = all_bonds

bonds.color[:] = np.stack(
    [fresnel.color.linear(bond_colors), fresnel.color.linear(bond_colors)], 
    axis=1
)
bonds.radius[:] = [0.03] * N_bonds


# create box in fresnel
fresnel.geometry.Box(scene, freud_box, box_radius=0.008);

In [ ]:
scene.lights = fresnel.light.ring()
fresnel.pathtrace(scene, light_samples=10)

In [ ]:
view = fresnel.interact.SceneView(scene)
view

In [ ]:
triclinic = mb.Box(lengths=[1,1,1], angles=[30,30,30])
f_triclinic = mb_to_freud_box(triclinic)

In [ ]:
Lx = f_triclinic.Lx
Ly = f_triclinic.Ly
Lz = f_triclinic.Lz
xy = f_triclinic.xy
xz = f_triclinic.xz
yz = f_triclinic.yz
# Follow hoomd convention
box_matrix = np.array([[Lx, xy*Ly, xz*Lz],
                          [0, Ly, yz*Lz],
                          [0, 0, Lz]])
a_1, a_2, a_3 = box_matrix.T
#           F--------------H
#          /|             /|
#         / |            / |
#        D--+-----------E  |
#        |  |           |  |
#        |  |           |  |
#        |  |           |  |
#        |  C-----------+--G
#        | /            | /
#        |/             |/
#        A--------------B
# Translate A so that 0, 0, 0 is the center of the 
A = -(a_1 + a_2 + a_3)/2
B = A + a_1
C = A + a_2
D = A + a_3
E = A + a_1 + a_3
F = A + a_2 + a_3
G = A + a_1 + a_2
H = A + a_1 + a_2 + a_3

In [ ]:
from numpy import linalg as LA


u1 = B-A #beta
v1 = D-A

u2 = B-A #gamma
v2 = C-A

u3 = C-A #alpha
v3 = D-A

#u = np.array([1,0,0])
#v = np.array([0,1,0])

def angle(u,v):
    return np.rad2deg(np.arccos(np.dot(u,v)/(LA.norm(u)*LA.norm(v))))

print("angle3: ", f"{angle(u3, v3):.4f}")
print("angle1: ", f"{angle(u1, v1):.4f}")
print("angle2: ", f"{angle(u2, v2):.4f}")

for pt in [A, B, C, D, E, F, G, H]:
    print(pt)

In [ ]:
box_scene = fresnel.Scene()

pos_array = np.stack([np.array([0, 0, 0]), A, B, C, D, E, F, G, H])

N = pos_array.shape[0]

# create box in fresnel
fresnel.geometry.Box(box_scene, f_triclinic, box_radius=0.008);

# put a sphere at the origin
geometry = fresnel.geometry.Sphere(box_scene, N=N, radius=0.06)
geometry.position[:] = pos_array
geometry.material = fresnel.material.Material(roughness=1.0)
geometry.outline_width = 0.01

# use color instead of material.color
geometry.material.primitive_color_mix = 1.0
geometry.color[:] = [
    fresnel.color.linear(cpk_colors[1]),
    fresnel.color.linear(cpk_colors[2]),
    fresnel.color.linear(cpk_colors[3]),
    fresnel.color.linear(cpk_colors[4]),
    fresnel.color.linear(cpk_colors[5]),
    fresnel.color.linear(cpk_colors[6]),
    fresnel.color.linear(cpk_colors[7]),
    fresnel.color.linear(cpk_colors[8]),
    fresnel.color.linear(cpk_colors[9])
                    ]
                     

fresnel.pathtrace(box_scene, light_samples=10)

In [ ]:
view = fresnel.interact.SceneView(box_scene)
view